In [ ]:
################## IMPORT LIBRARIES ##################

import numpy as np
import random 
import pandas as pd
import os
from os.path import join as pjoin
import matplotlib.pyplot as plt 

In [ ]:
################## IMPORT MY MODULES ##################

import sys
sys.path.append('../src')
import helpers as hlp
import importlib
importlib.reload(hlp);

In [ ]:
np.random.seed(42)

# directory where the data bases are stored
datapath="/home/ubuntu/Data_guestxr2/"

In [ ]:
# ---------- SPEECH POOL ------------

# create df with paths to speech files
speech_dataset_path1 =  pjoin(datapath,'VCTK','wav48_silence_trimmed')
speech_dataset_path2 =  pjoin(datapath,'PTDB')
speech_dataset_path3 =  pjoin(datapath,'EARS')

# initialize empty list of files:
speech_pool = []

# fill the list of files with filenames from vctk data base:
database="VCTK"
for root, dirs, files in os.walk(speech_dataset_path1):
    for file in files:
        if file.endswith('.flac'):
            # decide which split based on a probability 
            speech_pool.append({'database_speech': database, 'speech_file_path': os.path.join(root, file)})

# fill the list of files with filenames from ptdb data base:
database="PTDB"
for root, dirs, files in os.walk(speech_dataset_path2):
    for file in files:
        # make sure the correct speech files are used (MIC directory)
        if ("/MIC/" in root) & (file.endswith('.wav')):
            # decide which split based on a probability 
            speech_pool.append({'database_speech': database, 'speech_file_path': os.path.join(root, file)})  


# fill the list of files with filenames from ears data base:
database="EARS"
for root, dirs, files in os.walk(speech_dataset_path3):
    for file in files:
        # make sure the correct speech files are used (MIC directory)
        if file.endswith(('slow.wav','fast.wav','highpitch.wav','lowpitch.wav','regular.wav')):
            # decide which split based on a probability 
            speech_pool.append({'database_speech': database, 'speech_file_path': os.path.join(root, file)})  


# # shuffle order
random.shuffle(speech_pool)

# turn list to data frame 
speech_pool = pd.DataFrame(speech_pool).reset_index(drop=True)
speech_pool.tail(200)
print(f"{len(speech_pool)=}")

# split speech pool into test, train and val (70/15/15)
speech_train=speech_pool[speech_pool.index<int(0.7*len(speech_pool))].reset_index(drop=True)
speech_test=speech_pool[(speech_pool.index>=int(0.7*len(speech_pool))) & (speech_pool.index<int(0.85*len(speech_pool)))].reset_index(drop=True)
speech_val=speech_pool[(speech_pool.index>=int(0.85*len(speech_pool))) & (speech_pool.index<len(speech_pool))].reset_index(drop=True)

print(f"{len(speech_train)=}")
print(f"{len(speech_test)=}")
print(f"{len(speech_val)=}")

In [ ]:
# ---------- LOAD RIR PAIRS POOL ------------
# Here we load the info about all generated rirs ("rir_info.csv")
# Those rirs are arranged in pairs (see column "pair_nr")

database="synth_rirs_new"
rir_path=pjoin(datapath,database)


rirs_unique=pd.read_csv(pjoin(rir_path,"rir_info.csv"),index_col=0).reset_index(drop=True)
rirs_unique = rirs_unique.loc[:, ~rirs_unique.columns.str.contains('^Unnamed')]
unique_values = rirs_unique['ir_file_name'].unique()

# make a column with a file path that includes current directory 
rirs_unique["ir_file_path"] = rirs_unique["ir_file_name"].apply(lambda x: pjoin(rir_path, x))
if "ir_clone_file_name" in rirs_unique.columns: # if the database contained "cloned" RIRs (same room, different position)
    rirs_unique["ir_clone_file_path"] = rirs_unique["ir_clone_file_name"].apply(lambda x: pjoin(rir_path, x))

rirs_unique_train = rirs_unique[rirs_unique["pair_nr"]<7000].reset_index(drop=True)
rirs_unique_test = rirs_unique[(rirs_unique["pair_nr"]>=7000) & (rirs_unique["pair_nr"]<8500)].reset_index(drop=True)
rirs_unique_val = rirs_unique[(rirs_unique["pair_nr"]>=8500) & (rirs_unique["pair_nr"]<10000)].reset_index(drop=True)

print(f"{len(rirs_unique_train)=}")
print(f"{len(rirs_unique_test)=}")
print(f"{len(rirs_unique_val)=}")

In [ ]:
# ----- FUNCTION TO EXPAND RIR PAIRS POOL (BY FORMING NEW PAIRS) -----
# we have a dataset of 10000 rir pairs with a predefined rt60diff 
# each rir has a "clone" meaning an rir from the same room but different src pos
# so taking those into account we have 20000 pairs
# now we use each pair twice (each rir can be applied either to style or to content)
# with this, we end up with 40000 rir pairs 
# (each data point is a pair of style and content audio)

# --------- finding pairs with rt60diff ~ N(0.5,02) --------
# for each rir in this dataset, find a few additional pairs
# (but the rt60 diff should be distributed according to gauss N(0.5,02))

def expand_rir_pairs(rirs_pairs_in):
    possible_pairs = []
    mean_rt60diff=0.6
    std_rt60diff=0.2
    N_pairsperir=10
    for idx, rt60 in rirs_pairs_in["rt60_set"].items():
        # Compute differences with all other rows
        diffs = np.abs(rirs_pairs_in["rt60_set"] - rt60)

        # Exclude self-comparison
        diffs[idx] = np.nan  

        # Compute probabilities using a gauss N(0.5,02)
        probabilities = np.exp(-((diffs - mean_rt60diff) ** 2) / (2 * std_rt60diff ** 2))

        # Normalize to get a valid probability distribution
        probabilities[np.isnan(probabilities)] = 0
        probabilities /= np.nansum(probabilities)
    
        # Choose one row based on the computed probabilities
        chosen_indices = np.random.choice(diffs.index, size=N_pairsperir, p=probabilities)

        possible_pairs.append((idx, *chosen_indices))

    # --------- creating an expanded list of pairs --------
    rows=[]
    for idx in range(0, len(rirs_pairs_in)):
        # Copy rows of the pair
        row0 = rirs_pairs_in.loc[idx].copy()

        # Loop over the range of N  
        for i in range(N_pairsperir-1):
            # Use the indices from possible_pairs[idx] to get the corresponding rows
            row = rirs_pairs_in.loc[possible_pairs[idx][i+1]].copy()  # i+1 because the first element is idx
            rows.append(row0.copy())
            rows.append(row)

    rirs_pairs_out = pd.DataFrame(rows)
    rirs_pairs_out=rirs_pairs_out.reset_index(drop=True)

    # Compute absolute difference between rows
    rt60diffs = rirs_pairs_out["rt60_set"].diff().abs()

    # Assign the same difference to both rows in each pair
    rirs_pairs_out["rt60_diff"] = rt60diffs  

    # But a pair starts every second index, so we are only interested in those
    rirs_pairs_out.loc[0::2, "rt60_diff"] =rirs_pairs_out.loc[1::2, "rt60_diff"].values
    # pair number was the name of the column in the original csv from a data set 
    # it tells us from which original pair the current pair is formed, so i will leave 
    # it just in case

    rirs_pairs_out = rirs_pairs_out.rename(columns={"pair_nr": "gen_pair_nr"})


    return rirs_pairs_out

In [ ]:
# ----- EXPAND TRAIN, TEST AND VAL SET ----- 
rirs_train = expand_rir_pairs(rirs_unique_train)
rirs_train["split"] = "train"

rirs_test = expand_rir_pairs(rirs_unique_test)
rirs_test["split"] = "test"

rirs_val = expand_rir_pairs(rirs_unique_val)
rirs_val["split"] = "val"

print(f"{len(rirs_train)=}")
print(f"{len(rirs_test)=}")
print(f"{len(rirs_val)=}")


In [ ]:
print("Dataframe containing rir pairs. The pairs are divided into test,train and split ((see column <split>))")
print("PAIR IS DEFINED BY TWO CONSECUTIVE ROWS idx & idx+1, where idx=range(0,N,2) (even rows start a pair)")
display(rirs_train.head(5))
display(rirs_test.head(5))
display(rirs_val.head(5))

In [ ]:
# plot distribution of rt60 differences in the pairs
plt.figure()
plt.hist(rirs_train["rt60_diff"],bins =20)
plt.title("distribution of rt60diff in the rir pairs pool")
plt.show()

# plot distribution of rt60 differences in the pairs
plt.figure()
plt.hist(rirs_train["rt60_set"],bins =20)
plt.title("distribution of rt60 in the rir pairs pool")
plt.show()

In [ ]:
# --------- CREATE METADATA FOR A DATASET BY COMBINING SPEECH AND RIRS --------

def assign_speech_files(rir,speech):
    num_extra_samples = len(rir) - len(speech)
    extra_samples = speech.sample(n=num_extra_samples, replace=True, random_state=42)
    speech_expanded = pd.concat([speech, extra_samples], ignore_index=True)
    rirs_speech= pd.concat([rir.reset_index(drop=True), speech_expanded.reset_index(drop=True)], axis=1,ignore_index=False)
    return rirs_speech 

rirs_speech_train= assign_speech_files(rirs_train,speech_train)
rirs_speech_test= assign_speech_files(rirs_test,speech_test)
rirs_speech_val= assign_speech_files(rirs_val,speech_val)

print(f"{len(rirs_speech_train)=}")
print(f"{len(rirs_speech_test)=}")
print(f"{len(rirs_speech_val)=}")

In [ ]:
print("Full dataset metadata (pairs of rirs with assigned speech). Separate for test, train and val:")

display(rirs_speech_train.head(5))
display(rirs_speech_test.head(5))
display(rirs_speech_val.head(5))

In [ ]:
# --------- SAVE FINAL DATASET METADATA --------

from datetime import datetime
date_tag = datetime.now().strftime("%d-%m-%Y--%H-%M")

dataset_metadata = pd.concat([rirs_speech_train, rirs_speech_test, rirs_speech_val], ignore_index=True)
dataset_metadata = dataset_metadata.reset_index(drop=True)
dataset_metadata["pair_idx"] = dataset_metadata.index // 2  # Integer division groups every two rows

# create pilot csv (less data for quick tests):
dataset_metadata_pilot=pd.concat([rirs_speech_train[rirs_speech_train.index<int(0.01*len(rirs_speech_train))], 
                                  rirs_speech_test[rirs_speech_test.index<int(0.01*len(rirs_speech_test))], 
                                  rirs_speech_val[rirs_speech_val.index<int(0.01*len(rirs_speech_val))]], ignore_index=True)

# save metadata to csv:
dataset_metadata.to_csv("../dataset-metadata/"+ date_tag + "_data_set_t60diff.csv")
dataset_metadata_pilot.to_csv("../dataset-metadata/"+ date_tag + "_data_set_t60diff_pilot.csv")

dataset_metadata.head(20)

In [ ]:
# Check saved database
df = pd.read_csv("../dataset-metadata/ds2_metadata_example.csv",index_col=0)
df.head(20)

### 1. What data was exactly used in training? 

a) Speech pool: 
- VCTK (88328 audio files) + PTBD data bases (4718 audio files) + EARS (7276 audio files)
- In total: 93046 audio files

b) RIR pool: 
- 20 000 RIRs (10 000 pairs) from different rooms with varying source-receiver distance 
- Exact procedure is described in the script "rir_dataset.ipynb" (commited february 2025)
- The difference between pairs of rirs is controlled - the absolute differerence rt60diff ~N(0.5,02)

c) Style transfer data set
- 252k (train) + 54k (train) + 54k (val) samples = 126k (train) + 27k (test) + 27k (val) pairs consisting of 2 audio files and 2 rirs each
- To be able to create 352k samples, both speech and rirs are repeated
- But there is a form of augmentation in training:
- Every 2s audio sample is chosen randomly from a longer file
- For each rir we also have a "clone" rir - from the same room but from a different source position, so they can be used interchangeably while training

- 1 data point -> inputs: s1r1, s2r2 and ground truth: s1r2 
- Each audio sample is 2s randomly chosen from the audio file


In [ ]:
# get rt60 difference between style and content
df["diff_rt60"] =df["rt60_true"].diff()
df["diff_rt30"] =df["rt30_meas"].diff()
df.loc[0::2, 'diff_rt60'] = df['diff_rt60'].shift(periods=-1)
df.loc[0::2, 'diff_rt30'] = df['diff_rt30'].shift(periods=-1)


plt.figure(figsize=(10,6))
plt.rcParams.update({'font.size': 8})
plt.subplot(2,3,1)
plt.hist(df["volume"], bins=20, edgecolor='black')
plt.title("Volume of the room")
plt.subplot(2,3,2)
plt.hist(df["rt60_true"], bins=20, edgecolor='black')
plt.title("RT60 set in simulation")
plt.subplot(2,3,3)
plt.hist(df["rt30_meas"], bins=20, edgecolor='black')
plt.title("RT30 estimated")
plt.subplot(2,3,4)
plt.hist(df["diff_rt60"], bins=20, edgecolor='black')
plt.title("RT60 difference between style and content")
plt.subplot(2,3,5)
plt.hist(df["diff_rt30"], bins=20, edgecolor='black')
plt.title("RT30 difference between style and content")
plt.tight_layout()
plt.show()


# Simulation with masp uses rt60 like this: 
# # Compute absorption coefficients for desired rt60 and room dimensions
# abs_walls,rt60_true = srs.find_abs_coeffs_from_rt(room, rt60)
# # Small correction for sound absorption coefficients:
# if sum(rt60_true-rt60>0.05*rt60_true)>0 :
#     abs_walls,rt60_true = srs.find_abs_coeffs_from_rt(room, rt60_true + abs(rt60-rt60_true))


In [ ]:
# REPETITIONS OF FILES ACROSS DATA BASE

print(len(df.groupby("speech_file_path").size()))

# How many times each file is repeated across the whole data base
grouped_speech = df.groupby("speech_file_path")
grouped_rir = df.groupby("ir_file_path")

# How many times the files are repeated 
plt.figure(figsize=(8,3))
plt.subplot(1,2,1)
plt.hist(grouped_speech.size(), bins=40, edgecolor='black')
plt.xticks([1,2,3,4,5,6,7,8,9,10])
plt.xlabel("Nr of times a file appears in the database")
plt.ylabel("Nr of files")
plt.title("Speech files")
plt.subplot(1,2,2)
plt.hist(grouped_rir.size(), bins=40, edgecolor='black')
plt.xticks(np.arange(10,55,10))
plt.xlabel("Nr of times a file appears in the database")
plt.title("RIR files")
plt.ylabel("Nr of files")
plt.tight_layout()
plt.show()


In [ ]:
# TRAIN-TEST LEAKAGE 
split_counts_speech = grouped_speech['split'].nunique()
split_counts_rir = grouped_rir['split'].nunique()

# How many times the files are repeated 
plt.figure(figsize=(8,3))
plt.subplot(1,2,1)
plt.hist(split_counts_speech, bins=40, edgecolor='black')
plt.xticks([1,2,3])
plt.xlabel("Nr of splits a file is used in")
plt.ylabel("Nr of files")
plt.title("Speech files")
plt.subplot(1,2,2)
plt.hist(split_counts_rir, bins=40, edgecolor='black')
plt.xticks([1,2,3])
plt.xlabel("Nr of splits a file is used in ")
plt.title("RIR files")
plt.ylabel("Nr of files")
plt.tight_layout()
plt.show()


In [ ]:
# check metadata in combination with a dataset class definition
sys.path.append('../src')

import helpers as hlp
import dataset as ds
from IPython.display import Audio

importlib.reload(hlp)
importlib.reload(ds)

config=hlp.load_config("../config/basic.yaml")
config["dataset_metadata"]="/home/ubuntu/joanna/CWUNET/dataset-metadata/ds2_metadata_example.csv"
config["split"]="train"
dataset=ds.DatasetReverbTransfer(config)
# get one data sample 
sContent, sStyle, sTarget, sAnechoContent, sAnechoStyle = dataset[2]

# playback for the data sample
audios=[sContent, sStyle, sTarget, sAnechoContent]
names=["sContent", "sStyle", "sTarget", "sAnechoContent"]

for i,audio in enumerate(audios):
    print(names[i])
    audio=audio.squeeze(0).cpu()
    display(Audio(audio,rate=48e3))

